 Import Necessary Libraries

In [37]:
# Import necessary libraries
import zipfile
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle

In [38]:
from PIL import Image, ImageFile
from torchvision import datasets, transforms, models
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import seaborn as sns
import pandas as pd

Mount Google Drive and Extract ZIP file

In [39]:
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Define the path to the ZIP file in your Google Drive
root_dir = 'top_100_species_images/top_100_species_images'

Load Images and Extract Labels

In [40]:
#### Load the dataset from CSV file
df = pd.read_csv('FungiCLEF2023_train_metadata_PRODUCTION.csv')

#### Find the top 100 species by number of examples
top_100_species = df['species'].value_counts().head(100).index.tolist()

#### Filter the dataframe to only include rows from the top 100 species
df_top_100 = df[df['species'].isin(top_100_species)]

#### Create a new dataframe with only the 'image_path' and 'class_id' columns
top_100_pairs = df_top_100[['image_path', 'species']]

#### Print the shape of the final dataframe
print(f"Final dataframe shape: {top_100_pairs.shape}")

unique_species_ids = top_100_pairs['species'].nunique()
print(f"Number of unique species: {unique_species_ids}")
print(top_100_species)

# Create a dictionary that maps each species to a unique integer label
species_labels = {species: index for index, species in enumerate(top_100_species)}

print(species_labels)

Final dataframe shape: (84969, 2)
Number of unique species: 100
['Trametes versicolor', 'Fomitopsis pinicola', 'Hypholoma fasciculare', 'Pluteus cervinus', 'Fomes fomentarius', 'Bjerkandera adusta', 'Auricularia auricula-judae', 'Coprinellus micaceus', 'Stereum hirsutum', 'Mycena galericulata', 'Plicaturopsis crispa', 'Cuphophyllus virgineus', 'Hygrocybe miniata', 'Clitocybe nebularis', 'Ganoderma applanatum', 'Daedaleopsis confragosa', 'Pleurotus ostreatus', 'Tremella mesenterica', 'Gymnopilus penetrans', 'Meripilus giganteus', 'Armillaria lutea', 'Amanita muscaria', 'Tubaria furfuracea', 'Leccinum scabrum', 'Neoboletus luridiformis', 'Imleria badia', 'Lycoperdon perlatum', 'Phaeolepiota aurea', 'Trametes hirsuta', 'Trametes gibbosa', 'Boletus edulis', 'Pseudocraterellus undulatus', 'Byssomerulius corium', 'Schizophyllum commune', 'Kuehneromyces mutabilis', 'Xylaria hypoxylon', 'Daedalea quercina', 'Laccaria amethystina', 'Xerocomellus chrysenteron', 'Coprinus comatus', 'Phlebia treme

In [41]:
from torchvision import transforms

# Define transformations to be applied to each image (resizing, converting to tensor, normalizing)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 288x288 pixels
    transforms.ToTensor(),  # Convert image to a tensor (used by PyTorch)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [42]:
import os
from PIL import Image
from torch.utils.data import Dataset

# Define a custom dataset class for loading our images and labels
class CustomDataset(Dataset):
    def __init__(self, root_dir, top_100_df, transform=None):
        self.root_dir = root_dir
        self.top_100_df = top_100_df
        self.transform = transform
        self.image_paths = []  # List to store image paths
        self.labels = []       # List to store image labels
        image_count = 0
        label_count = 0
        iterc = 0

        print("Root directory:", root_dir)  # Print the root directory

        # Loop through each file in the root directory
        for file_name in os.listdir(root_dir):
            if file_name.endswith('.JPG') and not file_name.startswith('__MACOSX/'):

                if(iterc % 5 == 0):
                    # extract image file route
                    image_path = os.path.join(root_dir, file_name)
                    self.image_paths.append(image_path)  # Store the image path
                    image_count += 1

                    # Extract the label from the file name
                    image_path_after_slash = os.path.basename(file_name)
                    label = species_labels.get(self.top_100_df.loc[self.top_100_df['image_path'] == image_path_after_slash, 'species'].values[0])
                    self.labels.append(label)  # Store the corresponding label
                    label_count += 1

                iterc += 1

                if (iterc % 100) == 0:
                    print(iterc)

        print(len(set(self.labels)))
        print("Root directory:", root_dir)  # Print the root directory
        print("Image count:", image_count)
        print("Label count:", label_count)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [43]:
# Create dataset and dataloader objects for the training dataset
dataset = CustomDataset(root_dir=root_dir, top_100_df=top_100_pairs, transform=transform)

# Create DataLoaders to efficiently load data in batches
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

Root directory: top_100_species_images/top_100_species_images
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17

In [ ]:
# Define the model using pre-trained ResNet-50
class MushroomClassifier(nn.Module):
    def __init__(self, num_classes=100):
        super(MushroomClassifier, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Instantiate the model and move it to GPU if available
num_classes = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MushroomClassifier(num_classes=num_classes).to(device)

[INFO] preparing model ...


In [9]:
# Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# KFold keresztvalidációs trainelése a modellnek

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch

# Function to reset model weights
def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

# Define the train_kfold function
def train_kfold(model, dataset, batch_size, num_folds, num_epochs, optimizer, criterion, device):
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    train_losses, val_losses, train_accuracies, val_accuracies, val_f1_scores = [], [], [], [], []
    epoch_train_losses, epoch_val_losses, epoch_train_accuracies, epoch_val_accuracies, epoch_val_f1_scores = [], [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"Fold {fold + 1}/{num_folds}")

        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)

        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

        model.apply(reset_weights)

        for epoch in range(num_epochs):
            print("Itt vagyok epochban:", epoch + 1)
            model.train()
            epoch_train_loss, epoch_train_acc = 0.0, 0.0

            for batch_idx, (inputs, targets) in enumerate(train_loader):

                print(f"Fold: {fold + 1}, epoch: {epoch + 1}, train batch: {batch_idx + 1}")
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                epoch_train_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                epoch_train_acc += torch.sum(preds == targets).item()

                print(f"Loss in batch: {loss.item()}")
                print(f"Accuracy in batch: {torch.sum(preds == targets).item()}")

                train_losses.append(loss.item())
                train_accuracies.append(torch.sum(preds == targets).item())

            epoch_train_loss /= len(train_loader)
            epoch_train_acc /= len(train_idx)

            print(f"Train loss: {epoch_train_loss}")
            print(f"Train accuracy: {epoch_train_acc}")

            model.eval()
            epoch_val_loss, epoch_val_acc = 0.0, 0.0
            all_preds, all_targets = [], []

            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(val_loader):

                    print(f"Epoch: {epoch + 1}, validation batch: {batch_idx + 1}")
                    inputs, targets = inputs.to(device), targets.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    epoch_val_loss += loss.item()

                    _, preds = torch.max(outputs, 1)
                    epoch_val_acc += torch.sum(preds == targets).item()

                    all_preds.extend(preds.cpu().numpy())
                    all_targets.extend(targets.cpu().numpy())

                    val_losses.append(loss.item())
                    val_accuracies.append(torch.sum(preds == targets).item())

            epoch_val_loss /= len(val_loader)
            epoch_val_acc /= len(val_idx)
            epoch_val_f1 = f1_score(all_targets, all_preds, average='weighted')

            print(f"Validation loss: {epoch_val_loss}")
            print(f"Validation accuracy: {epoch_val_acc}")
            print(f"Validation F1 score: {epoch_val_f1}")

            print(f"Epoch {epoch + 1}/{num_epochs} | "
                  f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, "
                  f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}, Val F1: {epoch_val_f1:.4f}")

            epoch_train_losses.append(epoch_train_loss)
            epoch_val_losses.append(epoch_val_loss)
            epoch_train_accuracies.append(epoch_train_acc)
            epoch_val_accuracies.append(epoch_val_acc)
            epoch_val_f1_scores.append(epoch_val_f1)

    return train_losses, val_losses, train_accuracies, val_accuracies, val_f1_scores, epoch_train_losses, epoch_val_losses, epoch_train_accuracies, epoch_val_accuracies, epoch_val_f1_scores

In [11]:
# perform the first training session
train_losses, val_losses, train_accuracies, val_accuracies, val_f1_scores, epoch_train_losses, epoch_val_losses, epoch_train_accuracies, epoch_val_accuracies, epoch_val_f1_scores = train_kfold(
    model=model,
    dataset=dataset,
    batch_size= 32,
    num_folds= 2,
    num_epochs= 1,
    optimizer=optimizer,
    criterion=criterion,
    device=device
)

# save our model
torch.save(model.state_dict(), 'resnet50_model.pth')

Fold 1/2
Itt vagyok epochban: 1
Fold: 1, epoch: 1, train batch: 1
Loss in batch: 4.788154125213623
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 2
Loss in batch: 7.275820732116699
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 3
Loss in batch: 6.69547700881958
Accuracy in batch: 1
Fold: 1, epoch: 1, train batch: 4
Loss in batch: 6.218017578125
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 5
Loss in batch: 7.429649353027344
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 6
Loss in batch: 5.519186019897461
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 7
Loss in batch: 5.514373779296875
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 8
Loss in batch: 6.249794006347656
Accuracy in batch: 1
Fold: 1, epoch: 1, train batch: 9
Loss in batch: 5.3438825607299805
Accuracy in batch: 1
Fold: 1, epoch: 1, train batch: 10
Loss in batch: 6.266946792602539
Accuracy in batch: 0
Fold: 1, epoch: 1, train batch: 11
Loss in batch: 6.411480903625488
Accuracy in batch: 0
F

**Load the model again and perform further training sessions**

In [ ]:
# Load our model from the saved state dictionary
model = MushroomClassifier()
model.load_state_dict(torch.load('resnet50_model.pth'))
model.eval()

# Train further to finetune the model
train_losses, val_losses, train_accuracies, val_accuracies, val_f1_scores, epoch_train_losses, epoch_val_losses, epoch_train_accuracies, epoch_val_accuracies, epoch_val_f1_scores = train_kfold(
    model=model,
    dataset=dataset,
    batch_size= 32,
    num_folds= 3,
    num_epochs= 2,
    optimizer=optimizer,
    criterion=criterion,
    device=device
)

# Save it again
torch.save(model.state_dict(), 'resnet50_model.pth')

In [36]:
# Create a dictionary with the lists
data = {
    'train_losses': epoch_train_losses,
    #'val_losses': val_losses,
    'train_accuracies': epoch_train_accuracies,
    #'val_accuracies': val_accuracies,
}

print(len(train_losses))
print(len(val_losses))
print(len(train_accuracies))
print(len(val_accuracies))

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame as a CSV file
df.to_csv('training_results.csv', index=False)

2130
1066
2130
1066
